In [ ]:
%%capture

# Turn off warnings
import warnings

warnings.filterwarnings("ignore")

# Normal packages
import geopandas as gpd
import numpy as np
import pandas as pd

# Format
from babel.numbers import format_currency

# Display
from IPython.display import HTML, Image, Markdown, display, display_html

# Settings
pd.options.display.max_columns = 100
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)
pd.options.display.float_format = "{:,.2f}".format

# GCS, del later since this will presumbly be read from a script that cleans up the data
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/project_prioritization/"
FILE = "fake_data.xlsx"

# My utilities
import _utils

In [ ]:
# Charts
import altair as alt
from shared_utils import altair_utils
from shared_utils import calitp_color_palette as cp

In [ ]:
# Parameter Cell - the county of interest
parameter_county = "Inyo"

In [ ]:
# Create dfs
df_statewide = pd.read_excel(f"{GCS_FILE_PATH}{FILE}", sheet_name="fake")

In [ ]:
# Filter out a df for the county set in the parameter cell
df_parameter = df_statewide.loc[
    df_statewide["full_county_name"] == parameter_county
].reset_index(drop=True)

In [ ]:
# Grab the district this county is in
parameter_county_district = df_parameter["district"][0]

# Grab the full district name
parameter_county_district_full_name = df_parameter["district_full_name"][0]

In [ ]:
# Dataframe just for the district the parameter county is located in
df_parameter_district = df_statewide.loc[
    df_statewide["district"] == parameter_county_district
].reset_index(drop=True)

In [ ]:
df_parameter_district["district_full_name"][0]

In [ ]:
def county_district_comparison(df_parameter_county, df_parameter_district):
    # Grab the full district name
    district_full_name = df_parameter_district["district_full_name"][0]

    # Grab the full county name
    county_full_name = df_parameter_county["full_county_name"][0]

    # Create summary table for district
    district = _utils.summarize_by_project_names(df_parameter_district, "primary_mode")

    # Create summary table for county
    county = _utils.summarize_by_project_names(df_parameter_county, "primary_mode")

    # Append grand total and keep only that row...doesn't work when I try to do this with a for loop
    district = (
        district.append(district.sum(numeric_only=True), ignore_index=True)
        .tail(1)
        .reset_index(drop=True)
    )
    county = (
        county.append(county.sum(numeric_only=True), ignore_index=True)
        .tail(1)
        .reset_index(drop=True)
    )

    # Concat
    concat1 = pd.concat([district, county]).reset_index(drop=True)
    
    # Declare a list that is to be converted into a column
    geography = [district_full_name, county_full_name]
    concat1["Geography"] = geography
    
      
    # Divide to find % of county against district 
    concat1.loc[2] = (concat1.loc[1]/concat1.loc[0])*100

    # Drop old cols
    concat1 = concat1.drop(
        columns=[
            "Primary Mode",
            "Fake Fund Formatted",
            "Total Project ($1000) Formatted",
        ]
    )

    # Create new formatted monetary cols
    concat1["Total Project ($1000) Formatted"] = concat1[
        "Total Project Cost  $1,000"
    ].apply(lambda x: format_currency(x, currency="USD", locale="en_US"))

    concat1["Fake Fund Formatted"] = concat1["Current Fake Fund Requested"].apply(
        lambda x: format_currency(x, currency="USD", locale="en_US")
    )

    return concat1

In [ ]:
concat1 = county_district_comparison(df_parameter, df_parameter_district)

In [ ]:
concat1

In [ ]:
def repeated_charts(
    df,
    color_col: str,
    y_encoding_list: list,
    x_encoding_list: list,
    chart_title: str,
    tooltip_col: list,
):
    base = (
        alt.Chart()
        .mark_bar()
        .encode(
            color=alt.Color(
                color_col, scale=alt.Scale(range=cp.CALITP_DIVERGING_COLORS)
            ),
            tooltip= y_encoding_list + tooltip_col,
        )
        .properties(width=200, height=200)
        .interactive()
    )

    chart = alt.vconcat(data=df)
    for y_encoding in y_encoding_list:
        row = alt.hconcat()
        for x_encoding in x_encoding_list:
            row |= base.encode(x=x_encoding, y=y_encoding)
        chart &= row

    return chart.properties(title=chart_title)

In [ ]:
repeated_charts(
    concat1,
    "Geography:N",
    ["Geography:N"],
    [
        "Total Projects:Q",
        "Total Project Cost  $1,000:Q",
        "Current Fake Fund Requested:Q",
    ],
    "My Title",
    [ 
        "Total Project ($1000) Formatted",
        "Fake Fund Formatted",
        "Total Projects",
    ],
)